### Install dependencies

In [1]:
%env HDF5_USE_FILE_LOCKING=FALSE
# %env WANDB_MODE=dryrun
%env WANDB_NOTEBOOK_NAME=deepmr

env: HDF5_USE_FILE_LOCKING=FALSE
env: WANDB_NOTEBOOK_NAME=deepmr


In [3]:
import bpnet
from bpnet.cli.contrib import ContribFile
from bpnet.plot.tracks import plot_tracks, to_neg

import os
import uuid
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output, HTML
from pathlib import Path
import pandas as pd
import numpy as np
clear_output()

#### Optional: Setup wandb

In [4]:
import wandb

wandb.init(project="deepmr", entity="an1lam")

wandb: Currently logged in as: an1lam (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [5]:
# config variables
n_reps = 5

# file paths
config_dir = Path('./bpnet/') 

model_config_fname = 'ChIP-nexus-default.gin'
data_config_fname = 'ChIP-nexus.dataspec.yml'

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
project_dir = '/home/ubuntu/dev/an1lam/deepmr/'
data_dir = os.path.join(project_dir, 'dat')
output_dir = "/home/ubuntu/dev/an1lam/deepmr/dat/res-bpnet-training-2022-06-18-14-11-05"
# output_dir = os.path.join(data_dir, f'res-bpnet-training-{timestamp}')
output_dir

'/home/ubuntu/dev/an1lam/deepmr/dat/res-bpnet-training-2022-06-18-14-11-05'

In [6]:
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'output_ensemble'), exist_ok=True)

In [7]:
!cat {config_dir}/{data_config_fname}

task_specs:
  Oct4:
    tracks:
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.pos.bw
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.neg.bw
    peaks: /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/idr-optimal-set.summit.bed.gz
  Sox2:
    tracks:
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.pos.bw
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.neg.bw
    peaks: /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz
  Nanog:
    tracks:
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.pos.bw
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.neg.bw
    peaks: /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/idr-optim

### Data stats

In [8]:
# chromsomome names of differnet peaks
!zcat {project_dir}dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz \
    | cut -f 1 | sort -u

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chrX
chrY


Each task (or TF) can specify a set of peaks associated with it. Here are the number of peaks per TF we will use in this tutorial:

In [9]:
tasks = ['Oct4', 'Sox2', 'Nanog', 'Klf4']

# number of peaks per task
for task in tasks:
    print(task)
    !zcat {data_dir}/bpnet-manuscript-data/data/chip-nexus/{task}/idr-optimal-set.summit.bed.gz | wc -l

Oct4
25849
Sox2
10999
Nanog
56459
Klf4
57601


## 2. Train the model

Having specified `dataspec.yml`, we are now ready to train the model with 

```
bpnet train <dataspec.yml> <output dir> [optional flags]`
```

In [10]:
!cat {config_dir}/{model_config_fname} 
# NOTE: test_chr will be also excluded similar to 'exclude_chr'

b_loss_weight = 0
c_loss_weight = 10
p_loss_weight = 1
filters = 64
tconv_kernel_size = 25
lr = 0.004
n_dil_layers = 9
train.batch_size = 128
merge_profile_reg = False
dataspec = 'ChIP-nexus.dataspec.yml'

batchnorm = False

padding = 'same'
seq_width = 1000

tasks = ['Oct4', 'Sox2', 'Nanog', 'Klf4']


Have a look at the original gin file of bpnet9 here: https://github.com/kundajelab/bpnet/blob/master/bpnet/premade/bpnet9-ginspec.gin. For more information on using gin files see <https://github.com/google/gin-config>. 

To track model training and evaluation, we will use [wandb](http://wandb.com/) by adding `--wandb=avsec/bpnet-demo` to `bpnet train`. You can navigate to https://app.wandb.ai/avsec/bpnet-demo to see the training progress.

Let's train!

In [11]:
# setup all the file paths
example_model_dir = os.path.join(output_dir, 'output_ensemble', '0')

In [12]:
# Train for at most 10 epochs
for i in range(4, 5):
    # setup a new run_id (could be done automatically, but then the output directory would change)
    run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + "_" + str(uuid.uuid4())
    !cd {config_dir} && bpnet train {data_config_fname} --premade=bpnet9 \
        --config={model_config_fname} {output_dir} \
        --run-id '{run_id}' \
        --override='train.epochs=20; train.seed={i}'
    # softlink the new output directory
    !rm -rf {output_dir}/output_ensemble/{i} && ln -srf {output_dir}/{run_id} {output_dir}/output_ensemble/{i}

Using TensorFlow backend.

2022-07-04 18:47:02,067 [WARNING] From /home/ubuntu/anaconda3/envs/deepmr-bpnet3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2022-07-04 18:47:02,794 [INFO] NumExpr defaulting to 4 threads.
/home/ubuntu/src/bpnet/bpnet/plot/heatmaps.py:6: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  from mpl_toolkits.axes_grid1.colorbar import colorbar
INFO [07-04 18:47:04] Using gpu: 0, memory fraction: 0.45
2022-07-04 18:47:04.551078: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-07-04 18:47:04.680091: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2022-07-04 18:47:04.682003: I tensorflow/co

Epoch 1/20
2022-07-04 18:48:31.855315: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-07-04 18:48:37.980994: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
710/710 [==============================] - 308s 434ms/step - loss: 3452.2575 - Oct4/profile_loss: 855.8514 - Oct4/counts_loss: 1.0633 - Sox2/profile_loss: 467.3366 - Sox2/counts_loss: 0.3965 - Nanog/profile_loss: 1146.4181 - Nanog/counts_loss: 1.3698 - Klf4/profile_loss: 946.3728 - Klf4/counts_loss: 0.7983 - val_loss: 3343.9874 - val_Oct4/profile_loss: 830.7074 - val_Oct4/counts_loss: 0.7245 - val_Sox2/profile_loss: 465.3239 - val_Sox2/counts_loss: 0.2875 - val_Nanog/profile_loss: 1105.5917 - val_Nanog/counts_loss: 0.9673 - val_Klf4/profile_loss: 916.5757 - val_Klf4/counts_loss: 0.5995
Epoch 2/20
710/710 [==============================] - 272s 383ms/step - loss: 3268.9073 - Oct4/profile_loss:

710/710 [==============================] - 276s 388ms/step - loss: 3129.7291 - Oct4/profile_loss: 798.6716 - Oct4/counts_loss: 0.2724 - Sox2/profile_loss: 449.2429 - Sox2/counts_loss: 0.2061 - Nanog/profile_loss: 987.3892 - Nanog/counts_loss: 0.5743 - Klf4/profile_loss: 879.8261 - Klf4/counts_loss: 0.4070 - val_loss: 3186.4389 - val_Oct4/profile_loss: 809.9766 - val_Oct4/counts_loss: 0.2697 - val_Sox2/profile_loss: 458.2980 - val_Sox2/counts_loss: 0.1960 - val_Nanog/profile_loss: 1021.5029 - val_Nanog/counts_loss: 0.5532 - val_Klf4/profile_loss: 882.5261 - val_Klf4/counts_loss: 0.3946_loss: 793. - ETA: 50s - loss: 3129.8267 - Oct4/profile_loss: 799.0984
Epoch 14/20
710/710 [==============================] - 276s 388ms/step - loss: 3124.6255 - Oct4/profile_loss: 797.7637 - Oct4/counts_loss: 0.2650 - Sox2/profile_loss: 448.8865 - Sox2/counts_loss: 0.2017 - Nanog/profile_loss: 984.5245 - Nanog/counts_loss: 0.5550 - Klf4/profile_loss: 879.2332 - Klf4/counts_loss: 0.4000 - val_loss: 3180.07

Executing:  10%|███▏                           | 3/29 [00:04<00:42,  1.65s/cell]2022-07-04 20:31:57.661432: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-07-04 20:31:57.674443: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2022-07-04 20:31:57.674869: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55c9f4b188a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-04 20:31:57.674911: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-07-04 20:31:57.678061: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-07-04 20:31:57.771231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be 

In [13]:
! echo {output_dir}
! echo
! ls {output_dir}/output_ensemble/0

/home/ubuntu/dev/an1lam/deepmr/dat/res-bpnet-training-2022-06-18-14-11-05

bpnet-train.kwargs.json  events.out.tfevents.1655561487.ip-172-31-61-149
config.gin		 history.csv
config.gin.json		 input-config.gin
dataspec.yml		 log
evaluate.html		 model.h5
evaluate.ipynb		 note_params.json
evaluation.valid.json	 seq_model.pkl


In [14]:
!ls -latr {example_model_dir}/

total 14976
-rw-rw-r-- 1 ubuntu ubuntu       2 Jun 18 14:11 note_params.json
drwxrwxr-x 2 ubuntu ubuntu    4096 Jun 18 14:11 log
-rw-rw-r-- 1 ubuntu ubuntu     302 Jun 18 14:11 input-config.gin
-rw-rw-r-- 1 ubuntu ubuntu     561 Jun 18 14:11 bpnet-train.kwargs.json
-rw-rw-r-- 1 ubuntu ubuntu    1601 Jun 18 14:11 dataspec.yml
-rw-rw-r-- 1 ubuntu ubuntu    6034 Jun 18 14:11 config.gin
-rw-rw-r-- 1 ubuntu ubuntu    4191 Jun 18 14:11 config.gin.json
-rw-rw-r-- 1 ubuntu ubuntu    6448 Jun 18 15:36 history.csv
-rw-rw-r-- 1 ubuntu ubuntu 1741169 Jun 18 15:36 events.out.tfevents.1655561487.ip-172-31-61-149
-rw-rw-r-- 1 ubuntu ubuntu 1815880 Jun 18 15:36 model.h5
-rw-rw-r-- 1 ubuntu ubuntu 2318226 Jun 18 15:36 seq_model.pkl
-rw-rw-r-- 1 ubuntu ubuntu    8408 Jun 18 15:49 evaluation.valid.json
-rw-rw-r-- 1 ubuntu ubuntu 4572422 Jun 18 15:52 evaluate.ipynb
drwxrwxr-x 3 ubuntu ubuntu    4096 Jun 18 15:52 .
-rw-rw-r-- 1 ubuntu ubuntu 4811637 Jun 18 15:52 evaluate.html
drwxrwxr-x 9 ubuntu ubuntu    